# Build HF LLM

In [1]:
import torch
from langchain_huggingface import HuggingFacePipeline
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TextStreamer,
    pipeline,
)

# model_name = "MediaTek-Research/Breeze-7B-32k-Instruct-v1_0"
model_name = "meta-llama/Meta-Llama-3.1-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(
    model_name, cache_dir="../llm_model/", trust_remote_code=True
)
streamer = TextStreamer(tokenizer=tokenizer, skip_prompt=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    cache_dir="../llm_model/",
    device_map="auto",
    low_cpu_mem_usage=True,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    ),
)

pipe = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=True,
    max_new_tokens=2048,
    streamer=streamer,
)

hfPipeline = HuggingFacePipeline(pipeline=pipe)

e:\dev\auray\.venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct.
403 Client Error. (Request ID: Root=1-66b0d7c3-33edcdaf568b82ab56c3ab6e;c6b39969-65da-432d-bf77-6fbd55b5d576)

Cannot access gated repo for url https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct/resolve/main/config.json.
Your request to access model meta-llama/Meta-Llama-3.1-8B-Instruct is awaiting a review from the repo authors.

# Build vector database

In [1]:
import os

from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

embedding_cache_path = "../embedding_cache/"
db_collection_names = ["CNS16190_md", "CNS16190_md_only_provision"]
document_root_path = "../docs"
documents = ["CNS16190-zh_TW.md", "CNS16190-zh_TW_only_provision.md"]

collection = db_collection_names[0]
if os.path.isdir(os.path.join(embedding_cache_path, "collection", collection)):
    # database already exists, load it
    print("Loading existing database")
    vectorstore = Qdrant.from_existing_collection(
        embedding=OpenAIEmbeddings(),
        path=embedding_cache_path,
        collection_name=collection,
    )
else:
    # database does not exist, create it
    print("Creating new database")
    loader = UnstructuredMarkdownLoader(
        os.path.join(document_root_path, documents[0]), mode="elements"
    )
    doc = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(doc)

    vectorstore = Qdrant.from_documents(
        splits,
        embedding=OpenAIEmbeddings(),
        path=embedding_cache_path,
        collection_name=collection,
    )

retriever = vectorstore.as_retriever()


Loading existing database


In [6]:
question = """控制措施5.5-7，detail:消費者物聯網設備應保護透過遠端存取網路介面的關鍵安全參數的機密性。，answer:D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密方式進行，並採用最高等級的Cipher 符合FIPS 14--3 的安全強度模組進行，並且會隨時注意最先安全通報，檢查是否現行加密模組是否被發現漏洞，需要時即時更新。

  針對控制措施、detail和answer反推出疑問question"""
k = 10

print('similarity\n')
relevant_docs = vectorstore.search(question, search_type='similarity',k=k)

for item in relevant_docs:
    print(f'{item}\n')

print('\n\nsimilarity_score_threshold\n')
relevant_docs = vectorstore.search(question, search_type='similarity_score_threshold',k=k, score_threshold=0.7)

for item in relevant_docs:
    print(f'{item}\n')

print('\n\nmmr\n')
relevant_docs = vectorstore.search(question, search_type='mmr',k=k)

for item in relevant_docs:
    print(f'{item}\n')

similarity

page_content='控制措施5.5-7：消費者IoT裝置經由可遠端接取之網路介面進行通訊，應保護關鍵安全參數的機密性。' metadata={'source': '../docs\\CNS16190-zh_TW.md', 'category_depth': 2, 'last_modified': '2024-07-06T20:12:04', 'languages': ['kor'], 'parent_id': 'cde268301962cbb7ceb3f1969cd17bee', 'filetype': 'text/markdown', 'file_directory': '../docs', 'filename': 'CNS16190-zh_TW.md', 'category': 'Title', '_id': '9a26a5d614004d09a45a5dbcc7fcf88b', '_collection_name': 'CNS16190_md'}

page_content='控制措施5.5-7：消費者IoT裝置經由可遠端接取之網路介面進行通訊，應保護關鍵安全參數的機密性。' metadata={'source': '../docs\\CNS16190-zh_TW.md', 'category_depth': 2, 'last_modified': '2024-07-06T20:12:04', 'languages': ['kor'], 'parent_id': 'cde268301962cbb7ceb3f1969cd17bee', 'filetype': 'text/markdown', 'file_directory': '../docs', 'filename': 'CNS16190-zh_TW.md', 'category': 'Title', '_id': 'e2c073d36d174be0bd487f42e0f55bcf', '_collection_name': 'CNS16190_md'}

page_content='控制措施5.5-7：消費者IoT裝置經由可遠端接取之網路介面進行通訊，應保護關鍵安全參數的機密性。' metadata={'source': '../docs\\CNS16190-zh_TW.md

# Ask LLM

## Provision search

### Build rag chain

In [3]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

prompt.messages[
    0
].prompt.template = "你是一個資安專家，使用以下檢索到的背景資料回答問題，如果不知道答案就說不知道。\n背景資料：{context} \n問題：{question} \n答案："
prompt


ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='你是一個資安專家，使用以下檢索到的背景資料回答問題，如果不知道答案就說不知道。\n背景資料：{context} \n問題：{question} \n答案：'))])

In [4]:
from langchain_core.runnables import RunnablePassthrough

ls_run_name = "Provisino_Search_HF"
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} | prompt | hfPipeline
).with_config({"run_name": ls_run_name})


### Invoke LLM

In [5]:
questions = [
    "「若使用預先安裝之每裝置唯一通行碼，則應以機制產生此等通行碼，以降低對某類別或型式裝置的自動化攻擊之風險」符合哪一項控制措施？",  # 控制措施5.1-2
    "「對已揭露之脆弱性宜以及時方式採取行動」符合哪一項控制措施？",  # 控制措施5.2-2
    "「於所定義支援期間內，對其銷售及生產持續監視及矯正安全脆弱性。」符合哪一項控制措施？",  # 控制措施5.2-3（節錄）
    "「軟體以最小必要特殊權限運行」符合哪一項控制措施？",  # 控制措施5.6-7（節錄）
]


In [6]:
rag_chain.batch(questions)

e:\dev\auray\.venv\Lib\site-packages\transformers\models\mistral\modeling_mistral.py:674: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


控制措施5.1-2</s>
控制措施5.2-2</s>
控制措施5.2-3</s>
「軟體以最小必要特殊權限運行」符合控制措施5.6-7。</s>


["Human: 你是一個資安專家，使用以下檢索到的背景資料回答問題，如果不知道答案就說不知道。\n背景資料：[Document(page_content='控制措施5.1-2：若使用預先安裝之每裝置唯一通行碼，則應以機制產生此等通行碼，以降低對某類別或型式裝置的自動化攻擊之風險。', metadata={'source': '../docs\\\\CNS16190-zh_TW.md', 'category_depth': 2, 'last_modified': '2024-07-06T20:12:04', 'languages': ['kor'], 'parent_id': 'c1f23ff02f626c5074a9536b3e202419', 'filetype': 'text/markdown', 'file_directory': '../docs', 'filename': 'CNS16190-zh_TW.md', 'category': 'Title', '_id': 'eb1f51a42ddd4df29f1182f55e562c8c', '_collection_name': 'CNS16190_md'}), Document(page_content='控制措施5.1-2：若使用預先安裝之每裝置唯一通行碼，則應以機制產生此等通行碼，以降低對某類別或型式裝置的自動化攻擊之風險。', metadata={'source': '../docs\\\\CNS16190-zh_TW.md', 'category_depth': 2, 'last_modified': '2024-07-06T20:12:04', 'languages': ['kor'], 'parent_id': 'c1f23ff02f626c5074a9536b3e202419', 'filetype': 'text/markdown', 'file_directory': '../docs', 'filename': 'CNS16190-zh_TW.md', 'category': 'Title', '_id': 'a427786637d449d980ae90f5209bfd90', '_collection_name': 'CNS16190_md'}), Document(page_co

## Test cenario generation

### Build rag chain

In [7]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

prompt.messages[
    0
].prompt.template = "你是一個資安專家，使用以下檢索到的背景資料回答問題，如果不知道答案就說不知道。\n背景資料：{context} \n問題：{question} \n答案："
prompt


ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='你是一個資安專家，使用以下檢索到的背景資料回答問題，如果不知道答案就說不知道。\n背景資料：{context} \n問題：{question} \n答案：'))])

In [8]:
import pandas as pd


# concat all docs as a string
def format_docs(docs):
    retrieval_docs = "\n\n".join(doc.page_content for doc in docs)
    df = pd.read_csv("../docs/test_data_from_GoogleSheet.csv")
    test_data = "\n\n測試資料範例：\n\n"
    for idx in range(3):
        row = df.iloc[idx]
        test_data += f"控制措施{row['provision']}，detail:{row['detail']}，question:{row['test scenario']}，answer:{row['answer']}\n\n"

    return retrieval_docs + test_data


In [9]:
from langchain_core.runnables import RunnablePassthrough

ls_run_name = "Test_Scenario_Generation"
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | hfPipeline
).with_config({"run_name": ls_run_name})


### Invoke LLM

In [10]:
df = pd.read_csv("../docs/test_data_from_GoogleSheet.csv")

row = df.iloc[5]
query = f"控制措施{row['provision']}，detail:{row['detail']}，answer:{row['answer']}"
print(
    f"控制措施 {row['provision']}\n\n【detail】\n{row['detail']}\n\n【question】\n{row['test scenario']}\n\n【answer】\n{row['answer']}"
)

print(f"\n【question】\n{row['test scenario']}")

控制措施 5.4-1

【detail】
持久存儲中的敏感安全參數應由設備安全的存儲。

【question】
安全、可信任的儲存機制可用於保護敏感的安全參數。注意：此規定適用於持久性儲存（在裝置電源關閉後保留資料的任何資料儲存裝置），但製造商也可以對記憶體中的敏感安全參數實施類似的方法。問題：當設備有持久性儲存時：1. 設備如何確保敏感安全參數安全儲存？

【answer】
所有關鍵機敏資訊都以AES 128 加密演算法處理存儲在機器的flash中。

【question】
安全、可信任的儲存機制可用於保護敏感的安全參數。注意：此規定適用於持久性儲存（在裝置電源關閉後保留資料的任何資料儲存裝置），但製造商也可以對記憶體中的敏感安全參數實施類似的方法。問題：當設備有持久性儲存時：1. 設備如何確保敏感安全參數安全儲存？


In [11]:
rag_chain.invoke(f"{query}\n\n針對控制措施、detail和answer反推出疑問question")



1. 控制措施5.4-1的目的是什麼？

答案：控制措施5.4-1的目的是確保持久性儲存體中的敏感性安全參數由裝置安全儲存，以保護敏感數據並防止未經授權的訪問。

1. 什麼是持久性儲存體？

答案：持久性儲存體是指在設備上長期保存的數據和設置信息，包括敏感安全參數等信息。

1. 裝置安全儲存有哪些特點？

答案：裝置安全儲存是指在安全措施的保障下，對敏感安全參數等信息的存儲。通常包括以下特點：安全性、保密性、完整性、可用性、可恢復性。

1. 控制措施5.4-1如何保障敏感安全參數？

答案：控制措施5.4-1通過將持久性儲存體中的敏感安全參數由裝置安全儲存來保障敏感安全參數。具體措施可能包括使用安全演算法（如AES 128）進行加密、實施訪問控制等。

1. 如何報告設備製造商關於設備的安全漏洞？

答案：用戶可以通過設備製造商公開的漏洞公布政策，向其提供安全漏洞信息。政策通常包括報告問題的聯繫資訊和有關問題處理的時程表，如初始確認收貨和狀態更新等。

1. 在設備與雲端之間如何進行身份驗證？

答案：設備與雲端之間的身份驗證通常透過設備證書進行。在本機驗證方面，使用者設定的密碼透過HMAC-SHA256質詢完成。通信方面，本機驗證通常透過HTTPs TLS v1.2或更新的連線保護，雲與端通信方面也透過TLS v1.2或更新的連線進行保護。</s>


'Human: 你是一個資安專家，使用以下檢索到的背景資料回答問題，如果不知道答案就說不知道。\n背景資料：控制措施5.4-1：持久性儲存體中之敏感性安全參數應由裝置安全儲存。\n\n控制措施5.4-1：持久性儲存體中之敏感性安全參數應由裝置安全儲存。\n\n控制措施5.4-1：持久性儲存體中之敏感性安全參數應由裝置安全儲存。\n\n控制措施5.4-1：持久性儲存體中之敏感性安全參數應由裝置安全儲存。\n\n測試資料範例：\n\n控制措施5.1-1 ，detail:如果使用密碼並且處於出廠預設設置以外的任何狀態，則所有消費者物聯網設備密碼應為每個設備獨有的或由用戶定義。 ，question:問題：當使用密碼且處於出廠預設值（例如「admin」）以外的任何狀態時，所有消費者物聯網裝置密碼應符合下列其中一項：1. 每個設備都是唯一的。2. 由使用者定義。請詳細說明密碼是如何建立的。 ，answer:使用者密碼透過wizard由使用者設定且與MAC Address/SN無相關性，且無法反算或暴力計算。\n\n控制措施5.1-3，detail:用以對裝置鑑別使用者之鑑別機制，應使用對技術、風險及使用的性質適切之最佳實務密碼學。，question:與裝置所使用的身份驗證方法無關，身份驗證資料在使用者和裝置之間傳輸時應進行加密。注意：傳輸通常可以在行動應用程式和裝置之間進行。注意：密碼需要始終受到保護，加密是保護密碼的標準方法。 如果密碼未加密，則它們可以簡單地讀取（即明文）並且很容易洩漏。（範例：傳輸過程中的密碼保護通常可以透過 TLS/SSL 隧道連線或 VPN 進行）。問題：1. 當使用者向裝置進行身份驗證時，身份驗證資料在傳輸過程中如何進行加密保護？ ，answer:本機驗證透過 HMAC-SHA256 質詢使用使用者設定的密碼完成。 透過 HTTPs TLS v1.2 或更新的連線保護通道，chiper 套件符合 FIPS 14--3\r\n雲端管理通訊也透過 TLS v1.2 或更新的連線進行保護，chiper 套件符合 FIPS 14--3。 設備與雲端之間透過設備證書進行身份驗證。\n\n控制措施5.2-1，detail:製造商應公開漏洞公布政策。本政策至少應包括：報告問題的聯繫資訊;以及關於以下方面的時程表的資訊：1）收到初始確認;以及 2）狀態更新，直到報告